# Introduction
This notebook shows how to create a LangChain agent using a NASA Astronomy Photo of the Day tool, built using the `gofannon` library.
In this notebook, I'm going to:
- Load a tool that fetches NASA's Astronomy Picture of the Day (APOD) using their public API.
- Wrap that tool with LangChain so it can be used by an agent.
- Initialize a LangChain agnt using OpenAI's API
- Ask the agent natural language questions like "What's today's astronomy picture?"
- Have the agent retrieve and explain the image content using the APOD tool as its reasoning backend.

### What is `AstronomyPhotoOfTheDayTool`?

The `AstronomyPhotoOfTheDayTool` fetches data from NASA's **Astronomy Picture of the Day** API and returns:

- The image or video URL
- The title and explanation
- The date of the APOD


# Imports and setup
Install dependencies, import packages, and set up API keys.


In [31]:
!pip install langchain-openai langchain-community gofannon[langflow] --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Why These Libraries?
- `langchain-openai`: Integrates OpenAI models with LangChain for reasoning and decision-making.

- `langchain-community`: Core framework for building LangChain agents and workflows.

- `gofannon[langflow]`: Provides tools like NASA’s APOD API, which can be wrapped into LangChain for use in agents.

These libraries enable building an intelligent agent that can interact with external APIs and respond to user queries, and the `--quiet` keywords suppresses non-essential information and only shows warnings and errors.

# Setting API Keys

The following code may seem complex, but the gofannon project standardizes our examples on Google Colab because it provides users around the world with a consistent environment for experimentation, and when using "CPU Only" mode (which almost all of our examples are), it is free. So the following code will attempt to load environment secrets the Colab way, and if that fails, will attempt to load them from environment variables.

You as the user could also delete the code that follows and replace it with:

```python
openai_api_key = "<YOUR OPENAI KEY>"
nasa_api_key = "<YOUR NASA KEY>"


In [ ]:
import os
import requests
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

colab_present = False
try:
    from google.colab import userdata
    colab_present = True
except ImportError:
    pass

if colab_present:
    openai_api_key = userdata.get('open_ai_key')
    nasa_api_key = userdata.get('nasa_api_key')
else:
    openai_api_key = os.environ['OPENAI_API_KEY']
    nasa_api_key = os.environ['NASA_API_KEY']

# Load the tool
This example uses the NASA `AstronomyPictureOfTheDayTool` to fetch the Astronomy Picture Of The Day.
The APOD API provides daily space and astronomy related images along with short descriptions written by professional astronomers.

You can register for a free NASA API key here: https://api.nasa.gov/


In [ ]:

from gofannon.nasa.apod import AstronomyPhotoOfTheDayTool

apod_instance = AstronomyPhotoOfTheDayTool(api_key=nasa_api_key)
apod_tool = apod_instance.export_to_langchain()


# Initializing the LangChain agent

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-2024-08-06",
    openai_api_key=openai_api_key,
)

## Abut the LangChain Agent

This agent uses LangChain’s `initialize_agent` function with the `OPENAI_FUNCTIONS` type, which makes GPT-4o behave like a function-calling assistant.

The agent:
- Receives your question as input
- Decides whether to invoke the APOD tool
- Uses the tool’s response as context
- Replies with a final, coherent answer

In [ ]:
# Register the agent with the tool
agent = initialize_agent(
    tools=[apod_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)

# Running a Query


In [ ]:
query = "What is the Astronomy Photo of the Day?"
response = agent.run(query)

print(response)